In [2]:
import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write
import pyaudio
import wave

# Define threshold for voice detection
THRESHOLD = 500

# Recording parameters
fs = 44100  # Sample rate
chunk_size = 1024  # Size of audio chunks
duration = 5  # Maximum recording duration in seconds

# Initialize PyAudio
p = pyaudio.PyAudio()

# Get default input device
input_device_index = None
for i in range(p.get_device_count()):
    info = p.get_device_info_by_host_api_device_index(0, i)
    if info["maxInputChannels"] > 0:
        input_device_index = i
        break

# Create stream
stream = p.open(
    input_device_index=input_device_index,
    format=pyaudio.paInt16,
    channels=1,
    rate=fs,
    input=True,
    frames_per_buffer=chunk_size,
)

# Start recording
recording = []
recording_started = False
print("Recording...")
for i in range(int(fs * duration / chunk_size)):
    data = stream.read(chunk_size)
    rms = np.sqrt(np.mean(np.square(np.frombuffer(data, dtype=np.int16))))

    # Start recording if voice is detected
    if rms > THRESHOLD and not recording_started:
        print("Recording started!")
        recording_started = True
        recording.append(data)

    # Continue recording if voice is active
    if recording_started:
        recording.append(data)

    # Stop recording if voice is silent for a short time
    if recording_started and rms < THRESHOLD:
        print("Recording stopped!")
        break

# Combine recorded chunks
audio = np.concatenate([np.frombuffer(chunk, dtype=np.int16) for chunk in recording])

# Save audio to WAV file
print("Saving audio to file...")
write("recording.wav", fs, audio)
print("Recording saved successfully!")

# Close stream and PyAudio
stream.stop_stream()
stream.close()
p.terminate()

Recording...


C:\Users\niran\AppData\Local\Temp\ipykernel_3208\2507547388.py:42: RuntimeWarning: invalid value encountered in sqrt
  rms = np.sqrt(np.mean(np.square(np.frombuffer(data, dtype=np.int16))))


ValueError: need at least one array to concatenate